In [2]:
from pcdet.config import cfg_from_yaml_file, cfg
from pcdet.utils import common_utils
from pcdet.models import build_network
from pcdet.datasets import DatasetTemplate
from pathlib import Path
import numpy as np
import glob

In [3]:
cfg_file = "./cfgs/kitti_models/pointrcnn.yaml"

![data_vs_model](../docs/multiple_models_demo.png)

In [4]:
cfg_from_yaml_file(cfg_file=cfg_file, config=cfg)

{'ROOT_DIR': PosixPath('/home/ksas/chw_space/OpenPCDet_Developing'),
 'LOCAL_RANK': 0,
 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'],
 'DATA_CONFIG': {'DATASET': 'KittiDataset',
  'DATA_PATH': '../data/kitti',
  'POINT_CLOUD_RANGE': [0, -40, -3, 70.4, 40, 1],
  'DATA_SPLIT': {'train': 'train', 'test': 'val'},
  'INFO_PATH': {'train': ['kitti_infos_train.pkl'],
   'test': ['kitti_infos_val.pkl']},
  'GET_ITEM_LIST': ['points'],
  'FOV_POINTS_ONLY': True,
  'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'],
   'AUG_CONFIG_LIST': [{'NAME': 'gt_sampling',
     'USE_ROAD_PLANE': True,
     'DB_INFO_PATH': ['kitti_dbinfos_train.pkl'],
     'PREPARE': {'filter_by_min_points': ['Car:5',
       'Pedestrian:5',
       'Cyclist:5'],
      'filter_by_difficulty': [-1]},
     'SAMPLE_GROUPS': ['Car:20', 'Pedestrian:15', 'Cyclist:15'],
     'NUM_POINT_FEATURES': 4,
     'DATABASE_WITH_FAKELIDAR': False,
     'REMOVE_EXTRA_WIDTH': [0.0, 0.0, 0.0],
     'LIMIT_WHOLE_SCENE': True},
    {'NAME':

In [5]:
class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]

        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        if self.ext == '.bin':
            points = np.fromfile(self.sample_file_list[index], dtype=np.float32).reshape(-1, 4)
        elif self.ext == '.npy':
            points = np.load(self.sample_file_list[index])
        else:
            raise NotImplementedError

        input_dict = {
            'points': points,
            'frame_id': index,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict

In [9]:
data_path = "../../../Public/datasets/KITTI/training/velodyne/000000.bin"
ext = ".bin"

logger = common_utils.create_logger()
logger.info('-----------------Quick Demo of OpenPCDet-------------------------')

demo_dataset = DemoDataset(
        dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
        root_path=Path(data_path), ext=ext, logger=logger
    )

demo_dataset.class_names
demo_dataset.__len__()

2023-12-01 14:16:15,456   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2023-12-01 14:16:15,456   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2023-12-01 14:16:15,456   INFO  -----------------Quick Demo of OpenPCDet-------------------------


1

In [7]:
model = build_network(cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)